In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Variables pertaining to trained model

image_size = (224, 268)
cropped_image_size = (224, 224)
batch_size = 32
buffer_size = 4
test_path = './test_data'

# parse_image function returns an image and the image's label (which is given by the name of the folder in which
# the image is saved) given a filename
def parse_image(filename):
    parts = tf.strings.split(filename, os.sep)
    label = float(parts[-2])
    image = tf.io.read_file(filename)
    image = tf.image.decode_png(image)
    image = tf.image.resize(image, image_size)
    return image, label

# Construct a tensorflow Dataset based on the file list in the test_path parent directory
test_list_ds = tf.data.Dataset.list_files(str(test_path + os.sep + "*" + os.sep + "*.png")).shuffle(1000)
test_ds = test_list_ds.map(parse_image).batch(batch_size)
test_ds = test_ds.prefetch(buffer_size=buffer_size).cache()

print("Number of images in test dataset: ", test_list_ds.cardinality().numpy())

In [ ]:
# Load and display a summary of the trained model

model = keras.models.load_model('./simple_regression_trained_model')

model.summary()

In [ ]:
# Display a sample of the test data

%matplotlib inline

plt.figure(num=3, figsize=(20, 17))
for images, labels in test_ds.take(1):
    for i in range(25):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"), cmap='gray')
        plt.title(labels[i].numpy())
        plt.axis("off")
plt.show()

In [ ]:
# Evaluate the model on all the test data

score = model.evaluate(test_ds, verbose=1)
print("Test loss:", score)

In [ ]:
# Plot the prediction accuracy on an image-by-image basis - each datapoint represents a single image in the test data

labels = np.array([])
predictions = np.array([])
for x, y in test_ds:
    p = model.predict(x, verbose=1)
    for i in range(len(p)):
        predictions = np.concatenate([predictions, p[i]])
    labels = np.concatenate([labels, y.numpy()])

linear_model = np.polyfit(labels, predictions, 1)
linear_model_fn = np.poly1d(linear_model)
x_s = np.arange(4, 53)

plt.figure(num=2, figsize=(10, 10))
plt.title("Prediction Accuracy")
plt.plot(labels, predictions, 'o', markersize=3)
plt.plot(x_s, linear_model_fn(x_s), color="red")
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.show()

In [ ]:
# Plot the distribution of errors (the differences between predictions and true la)

errs = labels - predictions

plt.figure(num=4, figsize=(10, 10))
plt.title("Prediction Errors")
plt.hist(errs, bins=100)
plt.xlabel("Prediction Error")
plt.ylabel("Frequency")
plt.show()